In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
gdf_building = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/基盤地図/kiban03.shp')

In [4]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp')

In [7]:
gdf_osyu_100m_mesh[gdf_osyu_100m_mesh['Meshcode']=='5841515098']

,Meshcode,lu_code,lu_year,longitude,latitude,geometry
61814,5841515098,0700,20160811,141.135625,39.132917,"POLYGON ((26080.906 -96262.363, 26080.599 -961..."


In [5]:
src_proj = 4326  # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定

# 建物データの座標系変換
gdf_building.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_building = gdf_building.to_crs(epsg=dst_proj)  # 変換後座標に変換

# 100mメッシュデータの座標系変換
gdf_osyu_100m_mesh.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_osyu_100m_mesh = gdf_osyu_100m_mesh.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [ ]:
gdf_building

In [8]:
import time

# メッシュの面積を計算
gdf_osyu_100m_mesh['mesh_area'] = gdf_osyu_100m_mesh.geometry.area

# メッシュごとに処理を行うための空のリストを作成
building_ratio = []
# 各メッシュごとに処理を行う
total_time = 0
now_number = 0
for _, mesh_row in gdf_osyu_100m_mesh[61814:61815].iterrows():
    start_time = time.time()
    mesh_polygon = mesh_row['geometry']

    try:
        # メッシュと交差する建物の取得
        intersecting_buildings = gdf_building[gdf_building.intersects(mesh_polygon)]

        # メッシュと建物の交差部分の面積を計算
        intersection_areas = intersecting_buildings.intersection(mesh_polygon).area

        # メッシュの面積に対する建物の面積の比率を計算
        ratio = np.sum(intersection_areas) / mesh_row['mesh_area']

        # メッシュコードと建物の面積をリストに追加
        building_ratio.append(ratio)

    except Exception as e:
        # エラーが発生した場合はエラーメッセージを表示してスキップ
        print(f"エラーが発生しました：{e}")
        building_ratio.append(np.nan)
        continue

    end_time = time.time()
    elapsed_time = end_time - start_time
    total_time += elapsed_time
    now_number += 1

    # 処理時間の表示
    print(f"1回のループ処理の平均処理時間: {total_time / now_number}秒")
    print(f"ループ処理の回数: {now_number}回")


# geodataframeにリストを追加


1回のループ処理の平均処理時間: 0.6172935962677002秒
ループ処理の回数: 1回


In [9]:
building_ratio

[0.3836114937843611]

In [ ]:
gdf_osyu_100m_mesh = gdf_osyu_100m_mesh[50000:]

In [10]:
gdf_osyu_100m_mesh['building_ratio'] = building_ratio

ValueError: ignored

In [ ]:
gdf_osyu_100m_mesh[['Meshcode', 'building_ratio']].to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市 建物密集度/奥州市_建物密集度.csv')

In [ ]:
src_proj = 6678  # 変換前の座標系を指定
dst_proj = 4326  # 変換後の座標系を指定

# 建物データの座標系変換
gdf_building.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_building = gdf_building.to_crs(epsg=dst_proj)  # 変換後座標に変換

# 100mメッシュデータの座標系変換
gdf_osyu_100m_mesh.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_osyu_100m_mesh = gdf_osyu_100m_mesh.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [12]:
gdf_test = gdf_osyu_100m_mesh[gdf_osyu_100m_mesh['Meshcode']=='5841515098']
gdf_test['building_ratio'] = building_ratio

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [21]:
gdf_test[['Meshcode', 'mesh_area'	,'building_ratio']]

,Meshcode,mesh_area,building_ratio
61814,5841515098,9997.20245,0.383611


In [16]:
src_proj = 6678  # 変換前の座標系を指定
dst_proj = 4326  # 変換後の座標系を指定

# 建物データの座標系変換
gdf_test.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_test = gdf_test.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [18]:
src_proj = 4326  # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定

# 建物データの座標系変換
gdf_test.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_test = gdf_test.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [19]:
gdf_test.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市/奥州市　建物密集度/奥州市_建物密集度_確認.csv')